# Inference on the FPGA PYNQ-Z2 board

**NOTE**: This notebook does not run on CPU. This notebook is meant to run on a FPGA board. In this case, the [ZYNQ-Z2 board](https://www.amd.com/en/corporate/university-program/aup-boards/pynq-z2.html)

First, we import the necessary libraries, including the **hardware driver** required to manage the FPGA I/O interface. Low-level architectures such as FPGAs require explicit driver imports because data movement and register access are handled at the software–hardware boundary. Furthermore, these values change from board to board, depending on the architecture.

In [1]:
import numpy as np
from axi_lite_driver import AxiLiteDriverMuons

Now we can load a few examples, to test the inference:

In [2]:
#data = np.loadtxt("./FPGA_test/inference.csv", delimiter=",", skiprows=1)
X = np.loadtxt("./FPGA_test/x.csv", delimiter=",", skiprows=1)
y = np.loadtxt("./FPGA_test/y.csv", delimiter=",", skiprows=1)

#print(data.shape)   # (N_samples, N_features)

X = X[:, 1:]
y = y[:, -1]

print("Features shape:", X.shape)
print("Targets shape:", y.shape)

Features shape: (10, 27)
Targets shape: (10,)


We only took 10 examples, with 27 features

### Load overlay and prepare the accelerator

Now we are ready to load the **overlay**, a packaged hardware design (bitstream + metadata + Python/driver interface) that is loaded onto the programmable logic (PL) of an FPGA. It configures the FPGA fabric to implement a specific task, and exposes an API so software can interact with it.

The overlay is produced by Vivado HLS (now AMD Vitis HLS) using the hls4ml project created in the previous notebook `keras_to_hls.ipy

In [ ]:
from pynq import Overlay

overlay = Overlay("./FPGA_test/ai4hep.xsa")
#overlay.download()

# Check IP
print(overlay.ip_dict.keys())

In [ ]:
results = []
for i in range(len(X)):
    results.append(overlay.myproject_0.predict(X[i]))


Compare predictions vs target:

In [ ]:
import matplotlib.pyplot as plt

plt.scatter(y, results, s=10, alpha=0.7)
plt.xlabel("True target")
plt.ylabel("FPGA prediction")
plt.title("Regression: FPGA vs Ground Truth")
plt.grid(True)
plt.show()

And compute the RMSE:

In [ ]:
from sklearn.metrics import mean_squared_error

rmse = np.sqrt(mean_squared_error(y, results))

print("RMSE:", rmse)